In [1]:
#!pip install unidecode tqdm pandas

In [1]:
# First parse the data

In [2]:
def read_jsons(path_header = r'CORD-19-research-challenge/2020-03-13/'):
    import os, json
    import pandas as pd

    # this finds our json files
    path_to_json = path_header + r'biorxiv_medrxiv/biorxiv_medrxiv'
    path_to_json_2 = path_header + r"comm_use_subset/comm_use_subset"
    path_to_json_3 = path_header + r"noncomm_use_subset/noncomm_use_subset"
    path_to_json_4 = path_header + r"pmc_custom_license/pmc_custom_license"


    list_of_jsons= [path_to_json, path_to_json_2, path_to_json_3,path_to_json_4]

    json_files = []

    for i in list_of_jsons:
        json_files.extend([os.path.join(i,pos_json) for pos_json in os.listdir(i) if pos_json.endswith('.json')])



    #json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]


    # here I define my pandas Dataframe with the columns I want to get from the json
    jsons_data = pd.DataFrame(columns=['id', "title", "paper_abstract","paper_body"])

    # we need both the json and an index number so use enumerate()

    for index, js in enumerate(json_files):
        with open(js) as json_file:
            json_text = json.load(json_file)
            #print(json_text)
            # here you need to know the layout of your json and each json has to have
            # the same structure (obviously not the structure I have here)
            title = json_text['metadata']['title']

            paper_id = json_text["paper_id"]

            #reduces the list only if there is content
            if json_text["abstract"] != []:
                paper_abstract = (json_text["abstract"])
            else:
                paper_abstract = (json_text["abstract"])

            #paper_abstract = (json_text["abstract"])
            paper_body = (json_text["body_text"])
            #print(title)
            # here I push a list of data into a pandas DataFrame at row given by 'index'
            jsons_data.loc[index] = [paper_id, title, paper_abstract, paper_body]

    return jsons_data


In [3]:
jsons = read_jsons()

In [4]:
jsons

,id,title,paper_abstract,paper_body
0,a22ceac8d5e1cfaf2e79557d3c794458d0e1cdbe,23 of the 2019-nCoV sequence,"[{'text': 'In numerous instances, tracking the...",[{'text': '33 Meta-metagenomic searches allow ...
1,107f0d5a47f48f85d6645cac0e7008edc4d9d8cd,Quantitative Delineation of Herpesviruses in B...,[{'text': 'Public health concerns about recent...,[{'text': 'Public health concerns associated w...
2,681c5099b2afb08fe5cb2f58d92e9c6e4873bec2,Geometric coupling of helicoidal ramps and cur...,[{'text': 'Cellular membranes display an incre...,"[{'text': 'Recent advances in microscopy, segm..."
3,e4f66cfdaf836895e7d37941223f11ebc41dbb57,PLANT-Dx: A Molecular Diagnostic for Point of ...,[{'text': 'Synthetic biology based diagnostic ...,[{'text': 'Synthetic biology has recently cont...
4,e350e1b4aab99c73d0e0d03121b50c059c04fd61,Novel Immunoglobulin Domain Proteins Provide I...,[{'text': 'A novel coronavirus (SARS-CoV-2) is...,[{'text': 'Nidoviruses are an ancient group of...
...,...,...,...,...
13197,e29e17f37924d1d14edf56276a5dcca87851e62f,HIV-1 Frameshift RNA-Targeted Triazoles Inhibi...,[{'text': 'The HIV-1 frameshift-stimulating (F...,[{'text': 'T here is a rapidly expanding recog...
13198,6999b89c08c8a72fc0e86e78e4fd67666516a63a,,[{'text': 'Nosocomial infections are a major w...,[{'text': 'Nosocomial infections (NIs) are a m...
13199,16b0744424f02e01fe2f01b3ea03e2862f1359fc,Reference sequence (RefSeq) database at NCBI: ...,[{'text': 'The RefSeq project at the National ...,[{'text': 'For the past 15 years the National ...
13200,2d102f7db396b4358f23cab728f3af3d8871c39c,Protective Behavior and West Nile Virus Risk,[],"[{'text': 'We conducted a cross-sectional, hou..."


In [5]:
jsons.to_csv("jsons.csv")


In [1]:
#used this to buffer the extraction
import pandas as pd
jsons = pd.read_csv("jsons.csv")

In [2]:
jsons

,Unnamed: 0,id,title,paper_abstract,paper_body
0,0,a22ceac8d5e1cfaf2e79557d3c794458d0e1cdbe,23 of the 2019-nCoV sequence,"[{'text': 'In numerous instances, tracking the...",[{'text': '33 Meta-metagenomic searches allow ...
1,1,107f0d5a47f48f85d6645cac0e7008edc4d9d8cd,Quantitative Delineation of Herpesviruses in B...,[{'text': 'Public health concerns about recent...,[{'text': 'Public health concerns associated w...
2,2,681c5099b2afb08fe5cb2f58d92e9c6e4873bec2,Geometric coupling of helicoidal ramps and cur...,[{'text': 'Cellular membranes display an incre...,"[{'text': 'Recent advances in microscopy, segm..."
3,3,e4f66cfdaf836895e7d37941223f11ebc41dbb57,PLANT-Dx: A Molecular Diagnostic for Point of ...,[{'text': 'Synthetic biology based diagnostic ...,[{'text': 'Synthetic biology has recently cont...
4,4,e350e1b4aab99c73d0e0d03121b50c059c04fd61,Novel Immunoglobulin Domain Proteins Provide I...,[{'text': 'A novel coronavirus (SARS-CoV-2) is...,[{'text': 'Nidoviruses are an ancient group of...
...,...,...,...,...,...
13197,13197,e29e17f37924d1d14edf56276a5dcca87851e62f,HIV-1 Frameshift RNA-Targeted Triazoles Inhibi...,"[{'text': ""The HIV-1 frameshift-stimulating (F...",[{'text': 'T here is a rapidly expanding recog...
13198,13198,6999b89c08c8a72fc0e86e78e4fd67666516a63a,NaN,[{'text': 'Nosocomial infections are a major w...,[{'text': 'Nosocomial infections (NIs) are a m...
13199,13199,16b0744424f02e01fe2f01b3ea03e2862f1359fc,Reference sequence (RefSeq) database at NCBI: ...,[{'text': 'The RefSeq project at the National ...,"[{'text': ""For the past 15 years the National ..."
13200,13200,2d102f7db396b4358f23cab728f3af3d8871c39c,Protective Behavior and West Nile Virus Risk,[],"[{'text': 'We conducted a cross-sectional, hou..."


In [3]:
# Then we need to annotate that data

In [4]:
from unidecode import unidecode

def remove_non_ascii(text):
    return str(text).encode("ascii", errors="ignore").decode()

def analyze_text(text):
    import requests 
    # do some cleansing of this text    
    # defining the api-endpoint 
    API_ENDPOINT = 'http://nlu:8080/factextraction/analyze'
    headers = {"accept": "application/json", "content-type": "application/json"}
    # data to be sent to api
    #print("Text sent to ambiverse is: ", remove_non_ascii(text))
    data = r'{"docId": "doc2", "text":"' + remove_non_ascii(text) + r'", "extractConcepts": "true", "language": "en" }'
    # sending post request and saving response as response object 
    r = requests.post(url = API_ENDPOINT, data = data, headers = headers) 
    #print("returned text is: ", r.text)
    return r.text


In [5]:
# debugging only 
#from tqdm import tqdm
#import swifter
#jsons['out'] = jsons['title'].swifter.progress_bar(True).set_npartitions(npartitions=4).apply(analyze_text, axis=1)
#jsons['title_annotated'] = jsons.swifter.allow_dask_on_strings().progress_bar(True).set_npartitions(npartitions=8).apply(lambda row: analyze_text(row['title']), axis = 1)

#tqdm.pandas()
#jsons['title_annotated'] = jsons[0:12].apply(lambda row: analyze_text(row['title']), axis = 1) 

In [6]:
# adapted from here: https://www.kaggle.com/mlwhiz/parallelization-kernel
from multiprocessing import  Pool
import numpy as np
import pandas as pd


def parallelized_apply(df, func, numProcs=4):
    df_split = np.array_split(df, numProcs)
    pool = Pool(numProcs) 
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df
 

In [7]:
from tqdm import tqdm
tqdm.pandas()

def runAnnotate(df):
    # add here also the functions for all other columns
    df['title_annotated'] =  df.progress_apply(lambda row: analyze_text(row['title']), axis = 1)
    return df


jsons_annotated = parallelized_apply(jsons, runAnnotate, numProcs = 12) 

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 1101/1101 [36:27<00:00,  1.99s/it]


In [8]:
jsons_annotated

,Unnamed: 0,id,title,paper_abstract,paper_body,title_annotated
0,0,a22ceac8d5e1cfaf2e79557d3c794458d0e1cdbe,23 of the 2019-nCoV sequence,"[{'text': 'In numerous instances, tracking the...",[{'text': '33 Meta-metagenomic searches allow ...,"{""docId"":""doc2"",""language"":""en"",""matches"":[{""c..."
1,1,107f0d5a47f48f85d6645cac0e7008edc4d9d8cd,Quantitative Delineation of Herpesviruses in B...,[{'text': 'Public health concerns about recent...,[{'text': 'Public health concerns associated w...,"{""docId"":""doc2"",""language"":""en"",""matches"":[{""c..."
2,2,681c5099b2afb08fe5cb2f58d92e9c6e4873bec2,Geometric coupling of helicoidal ramps and cur...,[{'text': 'Cellular membranes display an incre...,"[{'text': 'Recent advances in microscopy, segm...","{""docId"":""doc2"",""language"":""en"",""matches"":[{""c..."
3,3,e4f66cfdaf836895e7d37941223f11ebc41dbb57,PLANT-Dx: A Molecular Diagnostic for Point of ...,[{'text': 'Synthetic biology based diagnostic ...,[{'text': 'Synthetic biology has recently cont...,"{""docId"":""doc2"",""language"":""en"",""matches"":[{""c..."
4,4,e350e1b4aab99c73d0e0d03121b50c059c04fd61,Novel Immunoglobulin Domain Proteins Provide I...,[{'text': 'A novel coronavirus (SARS-CoV-2) is...,[{'text': 'Nidoviruses are an ancient group of...,"{""docId"":""doc2"",""language"":""en"",""matches"":[{""c..."
...,...,...,...,...,...,...
13197,13197,e29e17f37924d1d14edf56276a5dcca87851e62f,HIV-1 Frameshift RNA-Targeted Triazoles Inhibi...,"[{'text': ""The HIV-1 frameshift-stimulating (F...",[{'text': 'T here is a rapidly expanding recog...,"{""docId"":""doc2"",""language"":""en"",""matches"":[{""c..."
13198,13198,6999b89c08c8a72fc0e86e78e4fd67666516a63a,NaN,[{'text': 'Nosocomial infections are a major w...,[{'text': 'Nosocomial infections (NIs) are a m...,"{""docId"":""doc2"",""language"":""en"",""matches"":[{""c..."
13199,13199,16b0744424f02e01fe2f01b3ea03e2862f1359fc,Reference sequence (RefSeq) database at NCBI: ...,[{'text': 'The RefSeq project at the National ...,"[{'text': ""For the past 15 years the National ...","{""docId"":""doc2"",""language"":""en"",""matches"":[{""c..."
13200,13200,2d102f7db396b4358f23cab728f3af3d8871c39c,Protective Behavior and West Nile Virus Risk,[],"[{'text': 'We conducted a cross-sectional, hou...","{""docId"":""doc2"",""language"":""en"",""matches"":[{""c..."


In [9]:
# save it to disk temporarly
jsons_annotated.to_csv('jsons_annotated.csv', index=True)

In [14]:
print(jsons_annotated['title_annotated'].iloc[0:1][0])

{"docId":"doc2","language":"en","matches":[{"charLength":8,"charOffset":20,"text":"sequence","entity":{"id":"http://www.wikidata.org/entity/Q863908","confidence":0.12579247315073913}}],"entities":[{"id":"http://www.wikidata.org/entity/Q863908","name":"Nucleic acid sequence","url":"http://en.wikipedia.org/wiki/Nucleic%20acid%20sequence","type":"CONCEPT","salience":0.0}]}


In [ ]:
# And ask the questions on it

In [13]:
print(jsons_annotated['title'].iloc[0:1][0])

23 of the 2019-nCoV sequence
